In [43]:
# Dependencies
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [44]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [45]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [46]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
#results = session.query(extract('month', Measurement.date))
#results = session.query(Measurement.date,Measurement.tobs).filter(extract('month', Measurement.date)==6)
results = session.query(Measurement.tobs).filter(extract('month', Measurement.date)==6)
#[row[1] for row in results.all()]
results

In [47]:
#  3. Convert the June temperatures to a list.
templist = [row[0] for row in results.all()]

type(templist)


list

In [48]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 

#added import pandas as pd at top
junetemp = pd.DataFrame(templist, columns=['junetemps'])


junetemp

,junetemps
0,78.0
1,76.0
2,78.0
3,76.0
4,77.0
...,...
1695,79.0
1696,74.0
1697,74.0
1698,76.0


In [49]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
df.describe()

,junetemps
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [50]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
results = session.query(Measurement.tobs).filter(extract('month',Measurement.date) == 12)

results

In [51]:
# 7. Convert the December temperatures to a list.
#technically .all()  already a list

results = [row[0] for row in results.all()]

In [52]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
dectemp = pd.DataFrame(results, columns=['dectemps'])

In [53]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
dectemp.describe()

,dectemps
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


In [54]:
#Code for dataframe desscribe merge found on 
#https://stackoverflow.com/questions/24563141/
#    merge-the-describe-method-output-in-pandas
#answered Jul 3, 2014 at 21:13
#CT Zhu
# WOW :)

summarydf = pd.merge(junetemp.describe(), 
                     dectemp.describe(), 
                     left_index=True, 
                     right_index=True, 
                     suffixes=('', '2'))

In [55]:
summarydf

,junetemps,dectemps
count,1700.000000,1517.000000
mean,74.944118,71.041529
std,3.257417,3.745920
min,64.000000,56.000000
25%,73.000000,69.000000
50%,75.000000,71.000000
75%,77.000000,74.000000
max,85.000000,83.000000


In [167]:
#Extra queries
# 1. Provide Accumulation of Precipitation
# 2. Provide Addition of March and September
# NOTE: a filter for just the Station 'USC00519281'
#      seems like it would be relevant since this is in coursework
# *. Additionally I included no of occurances of NO rain

marchrain = session.query(Measurement.date, Measurement.prcp).filter(extract('month', Measurement.date)==3).filter(Measurement.station == 'USC00519281')
junerain = session.query(Measurement.date, Measurement.prcp).filter(extract('month', Measurement.date)==6).filter(Measurement.station == 'USC00519281')
septrain = session.query(Measurement.date, Measurement.prcp).filter(extract('month', Measurement.date)==9).filter(Measurement.station == 'USC00519281')
decrain = session.query(Measurement.date, Measurement.prcp).filter(extract('month', Measurement.date)==12).filter(Measurement.station == 'USC00519281')


marchraindf = pd.DataFrame(marchrain, columns=['date','march_precip'])
marchraindf.set_index('date', drop=True, inplace=True)

juneraindf = pd.DataFrame(junerain, columns=['date','june_precip'])
juneraindf.set_index('date', drop=True, inplace=True)

septraindf = pd.DataFrame(septrain, columns=['date','sept_precip'])
septraindf.set_index('date', drop=True, inplace=True)

decraindf = pd.DataFrame(decrain, columns=['date','dec_precip'])
decraindf.set_index('date', drop=True, inplace=True)



tmp = { 
'Name' : ['march_precip', 'june_precip', 'sept_precip', 'dec_precip' ],
'amtprecip' : [marchraindf["march_precip"].sum(), juneraindf["june_precip"].sum(), 
                septraindf["sept_precip"].sum(), decraindf["dec_precip"].sum()],
'countnoprecip' : [int(marchraindf[marchraindf["march_precip"]==0].count()),
                    int(juneraindf[juneraindf["june_precip"]==0].count()), 
                    int(septraindf[septraindf["sept_precip"]==0].count()), 
                    int(decraindf[decraindf["dec_precip"]==0].count())]} 

precipitation_summary = pd.DataFrame(tmp, columns = ['Name', 'amtprecip', 'countnoprecip']) 

In [168]:
precipitation_summary

,Name,amtprecip,countnoprecip
0,march_precip,67.61,88
1,june_precip,35.76,45
2,sept_precip,46.77,46
3,dec_precip,53.15,50


In [162]:
#NVMD doesnt show much, counting yes rain is garbage, amount is better

# test = session.query(extract('year',Measurement.date)).filter(extract('month', Measurement.date)==6).filter(Measurement.station == 'USC00519281')
# approxnumberyears = len(np.unique(test.all()))
# approxnumberdays = 30
# approx_data_count = approxnumberyears * approxnumberdays

# approx_data_count

# print(marchraindf["march_precip"].count())
# print(juneraindf["june_precip"].count())
# print(septraindf["sept_precip"].count())
# print(decraindf["dec_precip"].count())
